In [1]:
import datetime
import hyp3_sdk
from tqdm import tqdm
from copy import deepcopy

## Setup

In [2]:
# uses .netrc; add `prompt=True` to prompt for credentials;
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu', prompt=True)
# hyp3_isce = hyp3_sdk.HyP3('https://hyp3-tibet-jpl.asf.alaska.edu', prompt=True)

In [3]:
failed_jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 3, 19), 
                                  end=datetime.datetime(2024, 1, 21),
                                  user_id='access_cloud_based_insar',
                                  status_code='FAILED')
dicts_failed = [j.to_dict() for j in failed_jobs]

print(failed_jobs)

65200 HyP3 Jobs: 0 succeeded, 65200 failed, 0 running, 0 pending.


In [7]:
jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 4, 29), 
                           end=datetime.datetime(2024, 5, 21))
print('#' * 10)
print(jobs)

##########
19075 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 19075 pending.


### topsapp

In [8]:
# Failed at isce2
topsapp_failed_jobs = [j for j in dicts_failed if len(j['processing_times']) == 1]
len(topsapp_failed_jobs)

19074

In [9]:
resubmit_dict = {'name': None,
                 'job_parameters': None,
                 'job_type': 'INSAR_ISCE'
                 }

prepared_jobs = []
for job in topsapp_failed_jobs:
    prepared_job = deepcopy(resubmit_dict)
    prepared_job['name'] = job['name']
    prepared_job['job_parameters'] = job['job_parameters']
    prepared_job['job_type'] = job['job_type']
    prepared_jobs.append(prepared_job)

In [59]:
batches_resubmit = [batch for batch in hyp3_sdk.util.chunk(prepared_jobs, 200)]
batches_resubmit[0][0], len(batches_resubmit[0]), len(batches_resubmit)

({'name': 'conus_west_coast-track115',
  'job_parameters': {'compute_solid_earth_tide': True,
   'esd_coherence_threshold': -1,
   'estimate_ionosphere_delay': True,
   'frame_id': 17887,
   'granules': ['S1A_IW_SLC__1SDV_20170211T141545_20170211T141615_015237_018F51_F3D3',
    'S1A_IW_SLC__1SSV_20170211T141516_20170211T141539_015237_018F50_B2FB'],
   'secondary_granules': ['S1A_IW_SLC__1SSV_20151219T141518_20151219T141545_009112_00D18D_9E27',
    'S1A_IW_SLC__1SSV_20151219T141542_20151219T141609_009112_00D18D_7814'],
   'weather_model': 'HRRR'},
  'job_type': 'INSAR_ISCE'},
 200,
 96)

In [77]:
### Submitting all the batches
print(f'Submitting: {len(prepared_jobs)} jobs')
#hyp3_jobs_out = [hyp3_isce.submit_prepared_jobs(batch) for batch in tqdm(batches_resubmit)]

Submitting: 19074 jobs


In [34]:
#reprocessing_date = datetime.datetime.today()
reprocessing_date = datetime.datetime(2024,5,1)
print(f'Date: {reprocessing_date}')
start_date = reprocessing_date - datetime.timedelta(days=1)
end_date = reprocessing_date + datetime.timedelta(days=1) 

print(hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                          job_type='INSAR_ISCE',
                          start=start_date, end=end_date))

Date: 2024-05-01 00:00:00
19074 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 19074 pending.


In [12]:
tjob = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                          job_type='INSAR_ISCE',
                          start=start_date, end=end_date)

### raider

In [87]:
# Failed at raider
raider_failed_jobs = [j for j in dicts_failed if len(j['processing_times']) == 2]
len(raider_failed_jobs)

45775

In [91]:
raider_failed_jobs[0]

{'job_id': '21862aa9-4e9c-436f-9a43-620b12f9e600',
 'job_type': 'INSAR_ISCE',
 'request_time': '2024-03-20T01:00:43+00:00',
 'status_code': 'FAILED',
 'user_id': 'access_cloud_based_insar',
 'name': 'conus_west_coast-track115',
 'job_parameters': {'compute_solid_earth_tide': True,
  'esd_coherence_threshold': -1,
  'estimate_ionosphere_delay': True,
  'frame_id': 17886,
  'granules': ['S1A_IW_SLC__1SDV_20230111T141536_20230111T141604_046737_059A5B_CFF5',
   'S1A_IW_SLC__1SDV_20230111T141602_20230111T141629_046737_059A5B_8B05'],
  'secondary_granules': ['S1B_IW_SLC__1SDV_20211217T141443_20211217T141510_030066_039705_D4FC',
   'S1B_IW_SLC__1SDV_20211217T141508_20211217T141535_030066_039705_A371'],
  'weather_model': 'HRRR'},
 'logs': ['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/21862aa9-4e9c-436f-9a43-620b12f9e600/21862aa9-4e9c-436f-9a43-620b12f9e600.log'],
 'expiration_time': '2024-10-29T00:00:00+00:00',
 'processing_times': [9109.396, 275.412],
 'credit

In [96]:
job_definition = {
    'name': 're-run RAiDER step',  # optional: provide a name
    "job_type": "ARIA_RAIDER",
    "job_parameters": {
        # "job_id": "27836b79-e5b2-4d8f-932f-659724ea02c3",
        # "weather_model": "ERA5"
    },
}

prepared_jobs = []
for job in raider_failed_jobs:
    prepared_job = deepcopy(job_definition)
    prepared_job['name'] = job['name'] + '_rerun_raider'
    prepared_job['job_parameters']['job_id'] = job['job_id']
    prepared_job['job_parameters']['weather_model'] = job['job_parameters']['weather_model'] 
    prepared_jobs.append(prepared_job)

In [107]:
import numpy as np
np.unique([j['name'] for j in prepared_jobs])

array(['conus_west_coast-track100_rerun_raider',
       'conus_west_coast-track108_rerun_raider',
       'conus_west_coast-track115_rerun_raider',
       'conus_west_coast-track137_rerun_raider',
       'conus_west_coast-track13_rerun_raider',
       'conus_west_coast-track144_1_rerun_raider',
       'conus_west_coast-track144_rerun_raider',
       'conus_west_coast-track166_1_rerun_raider',
       'conus_west_coast-track166_rerun_raider',
       'conus_west_coast-track173_rerun_raider',
       'conus_west_coast-track35_rerun_raider',
       'conus_west_coast-track42_rerun_raider',
       'conus_west_coast-track64_1_rerun_raider',
       'conus_west_coast-track64_rerun_raider',
       'conus_west_coast-track71_rerun_raider',
       'conus_west_coast-track86_rerun_raider',
       'conus_west_coast-track93_rerun_raider'], dtype='<U40')

In [98]:
raider_batches_resubmit = [batch for batch in hyp3_sdk.util.chunk(prepared_jobs, 200)]
raider_batches_resubmit[0][0], len(raider_batches_resubmit[0]), len(raider_batches_resubmit)

({'name': 'conus_west_coast-track115_rerun_raider',
  'job_type': 'ARIA_RAIDER',
  'job_parameters': {'job_id': '21862aa9-4e9c-436f-9a43-620b12f9e600',
   'weather_model': 'HRRR'}},
 200,
 229)

In [99]:
hyp3_isce.submit_prepared_jobs(prepared_jobs[0])

Batch([Job.from_dict({'job_id': '5309fa0b-393f-4de6-99d4-004569ad1a08', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-01T22:24:57+00:00', 'status_code': 'PENDING', 'user_id': 'access_cloud_based_insar', 'name': 'conus_west_coast-track115_rerun_raider', 'job_parameters': {'job_id': '21862aa9-4e9c-436f-9a43-620b12f9e600', 'weather_model': 'HRRR'}, 'credit_cost': 1})])

In [138]:
job = hyp3_isce.find_jobs(job_type='ARIA_RAIDER')
print(job)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [126]:
job.jobs[0].logs